# Predicción Precios Argentina - MD 2022

Notebook para la competencias de [Kaggle](https://www.kaggle.com/competitions/fcen-md-2022-prediccion-precio-de-propiedades/leaderboard) de la materia MD de la maestria de DM de la UBA.

Autor: Tomás Delvechio

# Configuración de la notebook

A continuación se definen configuraciones que afectaran el resto del notebook.

In [21]:
import sys
import os.path

In [22]:
EJECUTA_COLAB = 'google.colab' in sys.modules
#DATASET_RAW_LOCAL_NAME = 'ar_properties.csv'
#DATASET_COMPRESSED_LOCAL_NAME = DATASET_RAW_LOCAL_NAME + '.gz'


# Datasets

Se listan todos los datasets a considerar a continuación en un dict

In [28]:
datasets = {

    "input": {

        # Dataset de entrenamiento
        "entrenamiento": {
            "nombre": "properati_ar",
            "archivo": 'ar_properties.csv',
            "archivo_comprimido": "ar_properties.csv.gz",
            "comprimido": "gz",
            "url": "https://storage.googleapis.com/properati-data-public/ar_properties.csv.gz",
        },

        # Dataset de prueba
        "prueba": {
            "nombre": "testing",
            "archivo": "a_predecir.csv",
            "archivo_comprimido": "a_predecir.csv.zip",
            "comprimido": "zip",
            "url": "http://tomasdelvechio.github.io/subjects/dm/a_predecir.csv.zip",
        },

    },

    "output": {

        # Dataset para subir a Kaggle
        "solucion": {
            "nombre": "soluciones",
            "archivo": "solucion.csv",
            "comprimido": False,
            "url": None,
        },

    },

}

# Descarga de los datasets

Controla si los datasets estan o no descargado, en caso de no estarlo, los descarga y descomprime

In [36]:
if EJECUTA_COLAB:
    DOWNLOAD_PATH = '/content'
else:
    DOWNLOAD_PATH = '.data/'
    # tratamos de crear el dir por si es la 1era vez que ejecuta
    ! mkdir -p $DOWNLOAD_PATH

for tipo, ds in datasets["input"].items():
    print(f"Procesando ds para {tipo}: {ds['nombre']}")

    fulllpath = os.path.join(
        DOWNLOAD_PATH, f"{ds['archivo_comprimido']}")
    fulllpath_raw = os.path.join(
        DOWNLOAD_PATH, ds["archivo"])

    if os.path.exists(fulllpath_raw):
        # skip download
        print(f"Dataset {ds['nombre']} descargado... No se descarga...")
    else:
        url = ds["url"]
        print(f"Descargando {ds['nombre']} desde {url}")
        ! wget -N -O $fulllpath -q $url
        if ds["comprimido"] == "gz":
            ! gzip -d -f $fulllpath
        elif ds["comprimido"] == "zip":
            ! unzip -n $fulllpath -d ".data/"
            ! rm $fulllpath

archivo_entrenamiento = os.path.join(
    DOWNLOAD_PATH, datasets["input"]["entrenamiento"]["archivo"])
archivo_prueba = os.path.join(
    DOWNLOAD_PATH, datasets["input"]["prueba"]["archivo"])


Procesando ds para entrenamiento: properati_ar
Dataset properati_ar descargado... No se descarga...
Procesando ds para prueba: testing
Dataset testing descargado... No se descarga...


# 1. Importación de librerias

In [37]:
import pandas as pd
import sklearn as sk
from sklearn import model_selection
from sklearn import ensemble
from sklearn import metrics

# Lectura de datos

In [15]:
df_entrenamiento_inicial = pd.read_csv(archivo_entrenamiento, index_col="id")
df_prueba_inicial = pd.read_csv(archivo_prueba, index_col="id")


In [18]:
# filtro para pruebas rápidas
df = df.loc[(df.l2 == "Córdoba") & (df.operation_type == 'Venta') & (df.property_type == 'Casa')]
df.shape

(17972, 24)

# Análisis del conjunto de datos

# Tratamiento de los datos

# Modelización

In [19]:
df = df.select_dtypes(include=['float64', 'int64'])
df.fillna(0, inplace=True, downcast= "infer")

In [20]:
X = df[df.columns.drop('price')]
y = df['price']

for n_estimators in [100, 500, 1000]:     
    for max_depth in [3, 7, 11]:

        ## Tienen que usar RandomForestRegressor si o si o si. Pueden cambiar los parámetros
        reg = sk.ensemble.RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42, n_jobs=-1)

        ## Validación cruzada en 5 partes (lo van a ver en AA), -RMSE. No tocar
        scores = sk.model_selection.cross_val_score(reg, X, y, cv=10, scoring='neg_root_mean_squared_error')

        ## Imprimimos scores. Cuando más bajo mejor
        print(f"n_estimators={n_estimators}, max_depth={max_depth} --> {-scores.mean():.3f} +/- {scores.std():.3f}")


n_estimators=100, max_depth=3 --> 1092877.244 +/- 326348.764
n_estimators=100, max_depth=7 --> 1091396.303 +/- 303108.089
n_estimators=100, max_depth=11 --> 1057311.935 +/- 261268.926
n_estimators=500, max_depth=3 --> 1093316.561 +/- 326591.421
n_estimators=500, max_depth=7 --> 1090799.671 +/- 303480.374
n_estimators=500, max_depth=11 --> 1054131.309 +/- 261351.210
n_estimators=1000, max_depth=3 --> 1093700.609 +/- 326763.124
n_estimators=1000, max_depth=7 --> 1089087.818 +/- 300652.070
n_estimators=1000, max_depth=11 --> 1053844.945 +/- 261281.861


In [ ]:
df_pred = pd.read_csv('/content/a_predecir.csv', index_col="id")
df_pred.fillna(0, inplace=True)

In [ ]:
X_pred = df_pred[X.columns]

In [ ]:
# los mejores hiperparámetros encontrados antes
n_estimators = 100
max_depth = 3

# entrenamiento
reg = sk.ensemble.RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42, n_jobs=-1)
reg.fit(X, y)

# predicción
df_pred["price"] = reg.predict(X_pred)

In [ ]:
# grabo la solución
df_pred[["price"]].to_csv("solucion.csv")